In [ ]:
import geemap
import ee

Map = geemap.Map(center=[30, 112])
Map

In [ ]:
roi_file = r'H:/Monarch/paper/Data/jiujiang.shp'
point_file = r'H:/Monarch/paper/Data/landuse/points.shp'

In [ ]:
roi = geemap.shp_to_ee(roi_file)
psv = geemap.shp_to_ee(point_file)

In [ ]:
Map.addLayer(roi, {}, 'roi')
Map.addLayer(psv, {'color': 'FF0000'}, 'ps')

In [ ]:
from Monarch import user_gee
year = 2019
s2_year = []
for i in range(3, 12):
    s2 = ee.ImageCollection("COPERNICUS/S2")\
            .filterBounds(roi)\
            .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
            .filterDate(str(year), str(year+1))\
            .filter(ee.Filter.calendarRange(i, i, 'month'))
    s2_rm_cloud = s2.map(user_gee.rm_cloud_s2_sr)
    s2_z = s2.merge(s2_rm_cloud)
    a = s2.aggregate_array('system:time_start').map(lambda i: ee.Date(i).format('YYYY-MM-dd'))
    s2_year.append(s2_z.mosaic().select('B[1-7]').clip(roi).set('time', a.get(s2.size().subtract(1))))

In [ ]:
image = ee.ImageCollection(s2_year).toBands()

In [ ]:
image.bandNames().getInfo()

In [ ]:
Map.addLayer(image, {}, 'i')

In [ ]:
ps = psv.filter(ee.Filter.lte('CID', 50))
ps_y = psv.filter(ee.Filter.gt('CID', 50))

In [ ]:
ps.first().getInfo()

In [ ]:
label = 'RASTERVALU'

training = image.sampleRegions(**{
  'collection': ps,
  'properties': [label],
  'scale': 10
})
# 使用Cart(分类与回归树)
trained = ee.Classifier.smileCart().train(training, label)

In [ ]:
training.first().getInfo()

In [ ]:
result = image.classify(trained)

Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map

In [ ]:
a = psv.aggregate_array('RASTERVALU').getInfo()

In [ ]:
b = set(a)

In [ ]:
b